In [436]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [452]:
df = pd.read_excel('Data/bereinigte_DATEN_ano.xlsx')

In [453]:
df.head()

,Unnamed: 0,V1,athlete,gender,run,location,date,TurnNr,MinimalRadius,VelocityAtMinRadius,...,azitonextnext,HORIZONTALGATEDISTANCE,VERTICALGATEDISTANCE,regularbs1orfs2,steepness_A,filter_$,goofy2_regular1,@2Ferse_3Zehe_4Ferse,Zehen1_Fersen2,DistanzSchätz
0,0,35,3,1,1,Zermatt,200915,2,10.63,53.6,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
1,1,36,3,1,2,Zermatt,200915,2,11.05,52.3,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
2,2,37,3,1,3,Zermatt,200915,2,10.99,52.8,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
3,3,38,3,1,4,Zermatt,200915,2,11.41,52.2,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
4,4,39,5,1,1,Zermatt,200915,2,11.31,49.7,...,341.45477,8.515579,20.414966,1,2,0,2,3,1,30.635388


In [454]:
df_clean = df.drop(["Unnamed: 0", "V1", "date", "vectortonext", "vector2Dtonext", "vectortonextnorm", "vector2Dtonextnorm", "projpttonext", "vectortonextnext", "vector2Dtonextnext", "vectortonextnextnorm", "vector2Dtonextnextnorm", 'DistanzSchätz','Zehen1_Fersen2', 
                    '@2Ferse_3Zehe_4Ferse', 'filter_$', 'goofy2_regular1', 'athlete', 'run', 'Unnamed0', 'regularbs1orfs2'], axis=1)

In [455]:
# remove all time parameters
df_clean = df_clean.drop(["TimeStarttoEnd", "TimeStartToMinRad", "TimeMinRadToEnd", "GlideTime_endtostart", "TimeStarttoEnd_2"], axis =1)

In [456]:
df_clean.columns

Index(['gender', 'location', 'TurnNr', 'MinimalRadius', 'VelocityAtMinRadius',
       'VelocityAtTurnEntry', 'VelocityAtTurnExit', 'Initiation', 'Completion',
       'Gliding', 'GATEDISTANCE', 'dist2Dtonext', 'STEEPNESS', 'azitonext',
       'slopechangetonext', 'angletonext', 'projtonext', 'offsettonext',
       'dist3Dtonextnext', 'dts2Dtonextnext', 'slopetonextnext',
       'azitonextnext', 'HORIZONTALGATEDISTANCE', 'VERTICALGATEDISTANCE',
       'steepness_A'],
      dtype='object')

In [457]:
df.columns

Index(['Unnamed: 0', 'V1', 'athlete', 'gender', 'run', 'location', 'date',
       'TurnNr', 'MinimalRadius', 'VelocityAtMinRadius', 'VelocityAtTurnEntry',
       'VelocityAtTurnExit', 'TimeStarttoEnd', 'TimeStartToMinRad',
       'TimeMinRadToEnd', 'GlideTime_endtostart', 'TimeStarttoEnd_2',
       'Initiation', 'Completion', 'Gliding', 'Unnamed0', 'vectortonext',
       'vector2Dtonext', 'GATEDISTANCE', 'dist2Dtonext', 'vectortonextnorm',
       'vector2Dtonextnorm', 'STEEPNESS', 'azitonext', 'slopechangetonext',
       'angletonext', 'projtonext', 'projpttonext', 'offsettonext',
       'vectortonextnext', 'vector2Dtonextnext', 'dist3Dtonextnext',
       'dts2Dtonextnext', 'vectortonextnextnorm', 'vector2Dtonextnextnorm',
       'slopetonextnext', 'azitonextnext', 'HORIZONTALGATEDISTANCE',
       'VERTICALGATEDISTANCE', 'regularbs1orfs2', 'steepness_A', 'filter_$',
       'goofy2_regular1', '@2Ferse_3Zehe_4Ferse', 'Zehen1_Fersen2',
       'DistanzSchätz'],
      dtype='object')

In [458]:
possible_loc = df["location"].unique().tolist()
possible_loc

['Zermatt',
 'Stelvio',
 'Diavolezza',
 'Davos',
 'Kreuzbergpass',
 'Grindelwald',
 'Simonhöhe']

In [459]:
label_encoder = LabelEncoder()

In [460]:
df["location"] = label_encoder.fit_transform(df["location"])
df["location"]

0       6
1       6
2       6
3       6
4       6
       ..
6685    4
6686    4
6687    4
6688    4
6689    4
Name: location, Length: 6690, dtype: int64

In [461]:
possible_loc = df["location"].unique().tolist()
possible_loc

[6, 5, 1, 0, 3, 2, 4]

In [462]:
avg_MinRad = df["MinimalRadius"].mean()
avg_MinRad

14.16438565022423

In [496]:
possible_param = ['location', 'gender', 'MinimalRadius', 'VelocityAtMinRadius', 'VelocityAtTurnEntry',
       'VelocityAtTurnExit','STEEPNESS','HORIZONTALGATEDISTANCE', 'VERTICALGATEDISTANCE']

In [497]:
x = df[possible_param]
y = df[['TimeStarttoEnd_2']]

In [498]:
len(x.columns)

9

In [499]:
x.head()

,location,gender,MinimalRadius,VelocityAtMinRadius,VelocityAtTurnEntry,VelocityAtTurnExit,STEEPNESS,HORIZONTALGATEDISTANCE,VERTICALGATEDISTANCE
0,6,1,10.63,53.6,55.7,55.2,-16.13636,8.515579,20.414966
1,6,1,11.05,52.3,57.7,55.6,-16.13636,8.515579,20.414966
2,6,1,10.99,52.8,57.0,58.2,-16.13636,8.515579,20.414966
3,6,1,11.41,52.2,54.5,54.9,-16.13636,8.515579,20.414966
4,6,1,11.31,49.7,50.0,58.1,-16.13636,8.515579,20.414966


In [500]:
y.head()

,TimeStarttoEnd_2
0,1.52
1,1.56
2,1.56
3,1.76
4,2.08


In [501]:
# normalize the values
#normalized_x=(x-x.mean())/x.std()
#normalized_y=(y-y.mean())/y.std()

In [502]:
#normalized_x.head()

In [503]:
#normalized_y.head()

In [504]:
RANDOM_STATE=89389

In [505]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=RANDOM_STATE)

In [506]:
print(X_train.shape)
print(y_train.shape)

(5017, 9)
(5017, 1)


In [507]:
print(X_test.shape)
print(y_test.shape)

(1673, 9)
(1673, 1)


In [508]:
from sklearn.linear_model import LinearRegression

In [509]:
model = MultiOutputRegressor(LinearRegression()).fit(X_train, y_train)

In [510]:
model.score(X_test, y_test)

0.5207225247871385

In [511]:
model.predict(X_test)

array([[1.64935181],
       [1.60982103],
       [1.91569374],
       ...,
       [1.71459025],
       [1.18947364],
       [1.93230256]])

In [512]:
# new values for prediction
pred_location = 1
pred_gender = 1
pred_MinimalRadius = 10
pred_VelocityAtMinRadius = 100
pred_VelocityAtTurnEntry = 55
pred_VelocityAtTurnExit = 54
pred_HORIZONTALGATEDISTANCE = 9.5
pred_VERTICALGATEDISTANCE = 40
pred_STEEPNESS = -18

In [513]:
model.predict([[pred_location, pred_gender,pred_MinimalRadius,pred_VelocityAtMinRadius,
                pred_VelocityAtTurnEntry,pred_VelocityAtTurnExit,pred_HORIZONTALGATEDISTANCE,
                pred_VERTICALGATEDISTANCE, pred_STEEPNESS]])

array([[3.26219211]])